In [6]:
import nrrd
import numpy as np
from collections import deque
import glob
import nrrd
import nibabel as nib
import os
from scipy.ndimage import binary_dilation
from scipy.ndimage import label

In [7]:
def border_flood_fill_3d_optimized(image_3d, k):
    """
    Optimized border flood fill on a 3D image using scipy.ndimage.
    
    For each slice, identifies connected components of 0-valued pixels that 
    touch the border and replaces them with value k.
    
    Parameters:
    -----------
    image_3d : numpy.ndarray
        3D image array with shape (depth, height, width)
    k : int or float
        Value to replace 0-valued pixels with
        
    Returns:
    --------
    numpy.ndarray
        Modified 3D image array
    """
    result = image_3d.copy()
    depth, height, width = result.shape
    
    # Define 4-connectivity structure for 2D operations
    structure_2d = np.array([[0, 1, 0],
                             [1, 1, 1],
                             [0, 1, 0]], dtype=bool)
    
    # Process each slice
    for z in range(depth):
        current_slice = result[z]
        
        # Create binary mask of pixels with value 0
        zero_mask = (current_slice == 0)
        
        if not np.any(zero_mask):
            continue  # Skip if no zeros in this slice
            
        # Create border mask
        border_mask = np.zeros_like(zero_mask, dtype=bool)
        border_mask[0, :] = True      # Top row
        border_mask[-1, :] = True     # Bottom row
        border_mask[:, 0] = True      # Left column
        border_mask[:, -1] = True     # Right column
        
        # Find border zeros
        border_zeros = zero_mask & border_mask
        
        if not np.any(border_zeros):
            continue  # Skip if no border zeros
        
        # Find all connected components of zeros
        labeled_zeros, num_components = label(zero_mask, structure=structure_2d)
        
        # Get unique labels that touch the border
        border_labels = np.unique(labeled_zeros[border_zeros])
        border_labels = border_labels[border_labels > 0]  # Remove background label 0
        
        # Create mask for all border-connected zeros
        border_connected_mask = np.isin(labeled_zeros, border_labels)
        
        # Replace border-connected zeros with k
        current_slice[border_connected_mask] = k
    
    return result

In [ ]:
def border_flood_fill_3d(image_3d, k):
    """
    Performs border flood fill on a 3D image.
    
    For each slice, iterates through border pixels and changes pixels with value 0
    to value k, then flood-fills connected 0-valued neighbors.
    
    Parameters:
    -----------
    image_3d : numpy.ndarray
        3D image array with shape (depth, height, width)
    k : int or float
        Value to replace 0-valued pixels with
    
    Returns:
    --------
    numpy.ndarray
        Modified 3D image array
    """
    # Create a copy to avoid modifying the original image
    result = image_3d.copy()
    
    depth, height, width = result.shape
    
    # Define 4-connected neighbors (up, down, left, right)
    neighbors = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    def flood_fill_slice(slice_2d, start_row, start_col):
        """
        Flood fill starting from a border pixel in a 2D slice
        """
        if slice_2d[start_row, start_col] != 0:
            return
        
        # Use BFS for flood fill
        queue = deque([(start_row, start_col)])
        slice_2d[start_row, start_col] = k
        
        while queue:
            row, col = queue.popleft()
            
            # Check all 4-connected neighbors
            for dr, dc in neighbors:
                new_row, new_col = row + dr, col + dc
                
                # Check bounds
                if (0 <= new_row < height and 
                    0 <= new_col < width and 
                    slice_2d[new_row, new_col] == 0):
                    
                    slice_2d[new_row, new_col] = k
                    queue.append((new_row, new_col))
    
    # Process each slice
    for z in range(depth):
        current_slice = result[z]
        
        # Get border pixels for current slice
        border_pixels = []
        
        # Top and bottom rows
        for col in range(width):
            border_pixels.append((0, col))  # Top row
            if height > 1:
                border_pixels.append((height - 1, col))  # Bottom row
        
        # Left and right columns (excluding corners already added)
        for row in range(1, height - 1):
            border_pixels.append((row, 0))  # Left column
            if width > 1:
                border_pixels.append((row, width - 1))  # Right column
        
        # Process each border pixel
        for row, col in border_pixels:
            if current_slice[row, col] == 0:
                flood_fill_slice(current_slice, row, col)
    
    return result

def save_as_nifti(data, header, nifti_path):
    
    # Extract spacing information if available
    spacing = header.get('space directions', None)
    if spacing is not None:
        # Convert spacing to affine matrix
        affine = np.eye(4)
        for i in range(min(3, len(spacing))):
            if spacing[i] is not None:
                affine[i, i] = np.linalg.norm(spacing[i])
    else:
        # Default identity affine
        affine = np.eye(4)
    
    # Create and save NIfTI
    nifti_img = nib.Nifti1Image(data, affine)
    nib.save(nifti_img, nifti_path)


# Load a .nii.gz file
img = nib.load('path/to/your/file.nii.gz')

# Get the image data as a numpy array
data = img.get_fdata()

# Get image properties
print(f"Shape: {data.shape}")
print(f"Data type: {data.dtype}")
print(f"Header: {img.header}")
print(f"Affine matrix: {img.affine}")

In [ ]:
def remove_double_background(input_folder, output_folder):

    nii_files = glob.glob(f"{input_folder}\\*.nii.gz")
    files_found = len(nii_files)
    print(f"Found {files_found} .nii.gz files")
    progress_count = 0
    for file in nii_files:
        progress_count += 1
        print(f"Processing {progress_count} / {files_found}")
        if os.path.exists(file):
            file_name = os.path.basename(file)
            # file_name = file_name.replace('.nrrd', '.nii.gz') # for .nrrd
            output_path = os.path.join(output_folder, file_name)
        else:
            print(f"Cound not read {file}")
            continue
        
        #img_data, header = nrrd.read(file) # For .nrrd

        img = nib.load(file)
        img_data = img.get_fdata()

        processed_img = border_flood_fill_3d_optimized(img_data, -3200)

        new_img = nib.Nifti1Image(processed_img, img.affine, img.header)
        nib.save(new_img, output_path)

        #save_as_nifti(processed_img, header, output_path) # For .nrrd

input_folder = r"C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up\Follow_up"
output_folder = r"C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Follow_up_rmDoubleBg"

remove_double_background(input_folder, output_folder)
    

Found 148 .nii.gz files
Processing 1 / 148


FileNotFoundError: No such file or no access: 'path/to/your/file.nii.gz'